**Mail merge to send to a list - using OpenAI agents SDK**

In [ ]:
!pip install -q openai-agents brevo-python

In [ ]:
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel
from google.colab import userdata
import brevo_python #alt to sendgrid


In [ ]:
from brevo_python import ApiClient, Configuration
from brevo_python.api.transactional_emails_api import TransactionalEmailsApi
from brevo_python.models.send_smtp_email import SendSmtpEmail


In [ ]:
#Helper class for email sending
class EmailSender:
  def __init__(self, api_key) -> None:
    configuration = brevo_python.Configuration()
    configuration.api_key['api-key'] = api_key
    self.api_client = brevo_python.ApiClient(configuration)

  #todo: pass names too
  def send_email_full(self, frm, to, subject, body):
    email = SendSmtpEmail(
        to=[{"email": to, "name": "Raj"}],
        sender={"email": frm, "name": "Raju"},
        subject=subject,
        html_content=body
    )
    email_api = TransactionalEmailsApi(self.api_client)
    return email_api.send_transac_email(email)

  def send_email_short(self, subject=None, body=None):
    return self.send_email_full(frm="from@example.com", to="user1@example.com",
        subject=subject or "Test email from Brevo",
        body=body or "<html><body><h1>Hello!</h1><p>This is a test email.</p></body></html>"
    )


In [ ]:
#from typings import Dict    #not required. use builtin list/dict/tuple types.

@function_tool
def send_email(to_email_id:str, subject:str, body:str) -> str:
  try:
    brevo_key = userdata.get('BREVO_KEY')
    mail = EmailSender(brevo_key)
    mail.send_email_full(frm="from@example.com", to=to_email_id, subject=subject, body=body)
    return "success"
  except:
    return "fail" #log exception

@function_tool
def get_list() -> list[dict[str,str]]:
  '''returns collection of user names and their email ids'''
  return [{"name":"Raju","email_id":"user1@example.com"},{"name":"Raj","email_id":"user2@example.in"}]

@function_tool
def save_template(template:str):
  print(template)
  print("-----------")
  return "success"

In [ ]:
#without using liteLLM
model_name = "arcee-ai/trinity-large-preview:free"
base_url = "https://openrouter.ai/api/v1"

key = userdata.get('OPENROUTER_API_KEY')
open_router  = AsyncOpenAI(base_url=base_url, api_key=key)
model = OpenAIChatCompletionsModel(openai_client=open_router, model=model_name)

main_instruction = """
You are an email campaigner for a product, who send mail merge to a list.
Come up with good explanation on the product 'CatchU' - a trash picker gun.

Follow below steps carefully:
- Prepare email template with placeholder for user name.
- Write compelling and interesting post which attracts leads.
- Use HTML format with clean alignment.

Use below Tools in order:
- Call save_template tool to save the generated template for later use.
- Call the get_list tool to get the list of user name and their email Ids
- Call send_email tool to send email to each user.

Rules:
- you must generate single template and reuse it.
- Call the tools in specified order.
- Email body should be in HTML format.
"""
agent = Agent("campaigner", instructions=main_instruction, tools=[save_template, get_list, send_email], model=model)

message = "send campaign emails to users"

result = await Runner.run(agent, message)
print(result)

#Took 5 LLM calls - 4 tool_calls (1 save_template, 1 get_list, 2 send_email) + 1 final stop.
#"result" contains final output, explaining what it did in brief.